# Lab 3.4: Speech-to-Text with Whisper

**Objective**: Transcribe audio using OpenAI Whisper

**Duration**: 25 minutes

## Learning Outcomes
- Load Whisper model for transcription
- Handle audio files
- Detect language automatically

In [ ]:
import sys
sys.path.insert(0, "../../../src")
from hf_ecosystem import __version__
print(f"hf-ecosystem version: {__version__}")

In [ ]:
from transformers import pipeline
from datasets import load_dataset

## 1. Load Whisper Pipeline

In [ ]:
transcriber = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-tiny",
    device="cpu"
)
print("Whisper tiny model loaded")

## 2. Transcribe Audio

In [ ]:
# Load sample audio from dataset
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation[:1]")
audio = ds[0]["audio"]

print(f"Audio sampling rate: {audio['sampling_rate']} Hz")

In [ ]:
# Transcribe
result = transcriber(audio["array"])
print(f"Transcription: {result['text']}")

## Verification

In [ ]:
def verify_lab():
    assert "text" in result
    assert len(result["text"]) > 0
    print("✅ Lab completed successfully!")

verify_lab()